# Data Wrangling

## Librerias

### Instalacion

### Importacion

In [323]:
import re
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

## Carga Datos

In [296]:
dfFinalSet = pd.read_csv('FinalSet.csv', dtype = {'codigoencabezado': 'int32'})

dfFinalSet.describe(include = 'all')

,codigoencabezado,idregistro,linkpagina,fecharegistro,administracion,amueblado,antiguedad,banos,habitaciones,estudio,...,moneda,tipo,tipo_vendedor,favoritos,titulo,fecha_creacion,valido_hasta,descripcion,partner_code,user_id
count,2918.000000,2.918000e+03,2918,2918,612.000000,1779,1405,2916.000000,2904.000000,2904,...,2918,2915,2403,2918.000000,2918,2918,2918,2918,423,2.918000e+03
unique,NaN,NaN,1543,10,NaN,2,8,NaN,NaN,2,...,2,4,2,NaN,1450,1665,1528,1545,252,NaN
top,NaN,NaN,https://www.olx.com.gt/item/venta-apartamento-...,2020-06-10,NaN,No,Entre 5 y 10 años,NaN,NaN,No,...,US$,Apartamento,Inmobiliaria,NaN,ALQUILO APARTAMENTO EN VILLA CANALES,2020-05-03T13:47:49-06:00,2020-08-20T10:46:28-06:00,"Fácil acceso por Villa Nueva, Petapa, Villa Ca...",PVA-007-05-20-V,NaN
freq,NaN,NaN,11,550,NaN,1395,441,NaN,NaN,2887,...,1814,2849,2003,NaN,14,11,11,15,6,NaN
mean,1460.194311,1.095548e+09,NaN,NaN,808.816993,NaN,NaN,1.945816,2.308884,NaN,...,NaN,NaN,NaN,2.209390,NaN,NaN,NaN,NaN,NaN,4.043748e+07
std,842.866093,1.460999e+07,NaN,NaN,4543.787512,NaN,NaN,0.917046,0.810271,NaN,...,NaN,NaN,NaN,13.218336,NaN,NaN,NaN,NaN,NaN,2.521975e+07
min,1.000000,9.637780e+08,NaN,NaN,0.000000,NaN,NaN,0.000000,1.000000,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,4.263970e+06
25%,730.250000,1.100263e+09,NaN,NaN,0.000000,NaN,NaN,1.000000,2.000000,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,2.401211e+07
50%,1460.500000,1.100372e+09,NaN,NaN,0.000000,NaN,NaN,2.000000,2.000000,NaN,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,2.767485e+07
75%,2189.750000,1.100398e+09,NaN,NaN,650.000000,NaN,NaN,2.000000,3.000000,NaN,...,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,6.291165e+07


## Data Wrangling

En esta seccion prepararemos el set de datos que se utilizara para realizar la construccion del modelo de regresion. Primero definiremos los valores de flag para indicar que acciones se aplicaran sobre los datos.

In [297]:
# Normalizacion de datos para columna espacio_m2
flagDataNormalization = True

### Registros Unicos

Obtendremos solo los valores mas actualizados de los registros. Utilizaremos el atributo **idregistro** como identificador unico de los registros.

In [298]:
# Conversion a string del campo idregistro
dfAnalisisSet = dfFinalSet.astype({'idregistro': 'str'})

# Conversion a fecha del campo fecharegsitro
dfAnalisisSet['fecharegistro'] = pd.to_datetime(dfAnalisisSet['fecharegistro'], format = "%Y-%m-%d")

# Obtencion de fecha maxima por cada idregistro y filtro de esos registros.
dfAnalisisSet = dfAnalisisSet.loc[dfAnalisisSet.reset_index().groupby(['idregistro'])['fecharegistro'].idxmax()]

# Verificamos la cantidad de registros faltantes
dfAnalisisSet.shape

(1530, 26)

### Columna Precio Real

Procedemos a crear una columna llamada **precio_real** para establecer el precio en quetzales. Para los registros que tienen un precio en dolares se aplicara el factor de conversion a quetzales.

In [299]:
# Factor de conversion
cambio_moneda = 7.69

# Creacion de nueva columna
dfAnalisisSet['precio_real'] = np.where(dfAnalisisSet['moneda'] == 'US$', dfAnalisisSet['precio'] * cambio_moneda, 
                                        dfAnalisisSet['precio'])

# Visualizams los valores de precio real
dfAnalisisSet['precio_real']

1630    2299310.0
974     1207330.0
975      922800.0
965        6000.0
556        9612.5
          ...    
2515       4500.0
2514       6000.0
2513       9228.0
1645        900.0
2659    4921600.0
Name: precio_real, Length: 1530, dtype: float64

### Columna Oferta

Procederemos a crear una columna llamada oferta para determinar si es una venta o alquiler de apartamento. Para ello vamos a especificar un valor pivote en la columna precio. Todo valor superior al valor pivote sera considerado como una venta.

In [300]:
# Asignacion de valor pivote
intValorPivote = 50000

# Creacion de nueva columna oferta.
dfAnalisisSet['oferta'] = ['Venta' if precio > intValorPivote else 'Alquiler' for precio in dfAnalisisSet['precio_real']]

# Verificamos cuantos registros quedaron en el dataset
dfAnalisisSet.groupby(['oferta']).count()['codigoencabezado']

oferta
Alquiler    950
Venta       580
Name: codigoencabezado, dtype: int64

### Filtro Inicial

Los primeros filtros a aplicar a nuestro set de datos son los siguientes:
* Tipo de oferta como Venta, ya que solo nos interesa realizar el modelo para apartamentos en venta.
* Amueblado con valor No, ya que segun el analisis para las ventas no hay data para este atributo.
* El tipo se especificara como "Apartamento" ya que es la clasificacion que posee mayor cantidad de registros.

In [301]:
# Especificamos los filtros
dfFiltradoSet = dfAnalisisSet[(dfAnalisisSet['oferta'] == 'Venta') & 
                              ((dfAnalisisSet['amueblado'] == 'No') | (dfAnalisisSet['amueblado'].isnull())) &
                             (dfAnalisisSet['tipo'] == 'Apartamento')]

# Verificamos las dimensiones
dfFiltradoSet.shape

(562, 28)

Ahora especificamos las columnas que utilizaremos en principio para la construccion del set de datos

In [302]:
# Seleccionando las columnas a utilizar
dfFiltradoSet = dfFiltradoSet[['idregistro', 'banos', 'habitaciones', 
                               'espacio_m2', 'latitud', 'longitud', 'moneda',
                               'parqueo', 'tipo_vendedor', 'precio_real', 'user_id']]

dfFiltradoSet.shape

(562, 11)

### Nueva Variable Ubicacion (KMeans)

Se procedera a crear una nueva variable llamada ubicacion que sera el numero de cluster asignado dependiendo de los valores de longitud y latitud.

In [303]:
# Seleccion de numero de clusters.
# Nota: En Notebook Data Analysis podriamos identificar la cantidad de cluster por medio de elbow plot
intKClusters = 10

# Se obtienen los valores que se utilizaran para la construccion de modelo KMeans
npUbicaciones = np.array(list(zip(dfFiltradoSet['longitud'], 
                                  dfFiltradoSet['latitud']))).reshape(len(dfFiltradoSet['longitud']), 2)

# Construccion del modelo
kmeanModel = KMeans(n_clusters = intKClusters, max_iter = 2000).fit(npUbicaciones)

# Asignacion de clusters a registros
dfFiltradoSet['ubicacion'] =["U" + str(ubicacion) for ubicacion in kmeanModel.labels_]

# Verificacion de la nueva columna
dfFiltradoSet.head(10)

,idregistro,banos,habitaciones,espacio_m2,latitud,longitud,moneda,parqueo,tipo_vendedor,precio_real,user_id,ubicacion
1630,1007688385,3.0,3.0,232.0,14.591,-90.506,US$,NaN,Inmobiliaria,2299310.00,55074295,U0
974,1014321966,2.0,1.0,78.0,14.600,-90.511,US$,Si,Inmobiliaria,1207330.00,34408653,U0
975,1014324725,2.0,2.0,65.0,14.593,-90.491,US$,Si,Inmobiliaria,922800.00,34408653,U0
557,1023944274,2.0,2.0,74.0,14.588,-90.503,US$,Si,Inmobiliaria,1647474.84,24012106,U0
559,1029720345,2.0,3.0,290.0,14.607,-90.470,US$,Si,Inmobiliaria,5190750.00,24012106,U5
2219,1030164820,2.0,2.0,74.0,14.588,-90.503,US$,Si,Inmobiliaria,1647474.84,24012106,U0
521,1039739280,2.0,2.0,74.0,14.588,-90.503,US$,Si,Inmobiliaria,1647474.84,24012106,U0
526,1041192203,1.0,1.0,79.0,14.588,-90.503,Q,Si,Inmobiliaria,1329277.00,24012106,U0
527,1041192824,2.0,2.0,79.0,14.588,-90.503,Q,Si,Inmobiliaria,1604456.00,24012106,U0
528,1041193829,2.0,3.0,114.0,14.588,-90.503,Q,Si,Inmobiliaria,1604456.00,24012106,U0


### Filtro Baja Frecuencia (Ubicacion)

Verificaremos para la variable **ubicacion** del dataframe la frecuencia de sus valores para dejar aquellos que se encuentran bien representados.

In [304]:
srFreqUbicacion = pd.Series(dfFiltradoSet['ubicacion']).value_counts()
srFreqUbicacion

U0    272
U3    114
U5     83
U7     27
U2     17
U8     17
U4     15
U6     12
U9      4
U1      1
Name: ubicacion, dtype: int64

In [305]:
# Eliminando elementos que tienen baja frecuencia para ubicacion
intNumeroPivote = 5
idxEliminar = srFreqUbicacion[srFreqUbicacion <= intNumeroPivote].index
dfFiltradoSet = dfFiltradoSet[~dfFiltradoSet.ubicacion.isin(idxEliminar)]

### Manejo valores nulos

#### Validacion variables con valores nulos
Procedemos a verificar cuales son las variables que presentan valores nulos

In [306]:
dfFiltradoSet.isnull().sum()

idregistro         0
banos              0
habitaciones       0
espacio_m2         0
latitud            0
longitud           0
moneda             0
parqueo           14
tipo_vendedor    137
precio_real        0
user_id            0
ubicacion          0
dtype: int64

#### Tratamiento Valores Nulos Parqueo
Primero verificaremos los rangos de valores para la variable **parqueo**.

In [307]:
srFreqParqueo = pd.Series(dfFiltradoSet['parqueo']).value_counts(dropna = False)
srFreqParqueo

Si     446
No      97
NaN     14
Name: parqueo, dtype: int64

Vamos a utilizar la columna de descripcion para verificar si tiene la columna parqueo. De ser asi, se les colocara que si posee parqueo.

In [308]:
# Obtenemos la info de los registros que tienen parqueo como nulo.
dfSinParqueo = dfFiltradoSet.loc[pd.isnull(dfFiltradoSet['parqueo']), ['idregistro', 'parqueo']]

# Obtenemos la info de los regsitros del catalogo inicial solo para los registros de interes.
dfCatalogo = dfAnalisisSet.loc[dfAnalisisSet.idregistro.isin(dfSinParqueo['idregistro']), ['idregistro', 'descripcion']]

# Verificamos si existe palabra parqueo dentro de columna descripcion
dfCatalogo['parqueo'] = ["Si" if (re.search('parqueo',str(descripcion).lower())) else "No" \
                         for descripcion in dfCatalogo['descripcion']]

# Asignacion de los valores calculados para parqueo
dfFiltradoSet = dfFiltradoSet.set_index(['idregistro'])\
    .combine_first(dfCatalogo[['idregistro', 'parqueo']].set_index(['idregistro']))\
    .reset_index()


# Verificamos si existe palabra parqueo dentro de columna descripcion
dfFiltradoSet['parqueo'] = [1 if (parqueo == "Si") else 0 for parqueo in dfFiltradoSet['parqueo']]

# Verificacion del parqueo
srFreqParqueo = pd.Series(dfFiltradoSet['parqueo']).value_counts(dropna = False)
srFreqParqueo

1    457
0    100
Name: parqueo, dtype: int64

#### Tratamiento Valores Nulos Tipo Vendedor

In [309]:
srFreqVendedor = pd.Series(dfFiltradoSet['tipo_vendedor']).value_counts(dropna = False)
srFreqVendedor

Inmobiliaria     400
NaN              137
Dueño Directo     20
Name: tipo_vendedor, dtype: int64

In [310]:
# Obtenemos los codigos de vendedor donde el tipo_vendedor es nulo.
dfSinVendedor = dfFiltradoSet.loc[pd.isnull(dfFiltradoSet['tipo_vendedor']), ['idregistro', 'tipo_vendedor', 'user_id']]

# Calculamos la frecuencia de registros para cada user_id
srFreqVendedores = pd.Series(dfSinVendedor['user_id']).value_counts(dropna = False)

# Para aquellos user_id que solo poseen un registro se les asigna el valor de "Dueno Directo"
intNumeroPivote = 1
idxDuenos = srFreqVendedores[srFreqVendedores == intNumeroPivote].index
dfSinVendedor['tipo_vendedor'] = ["Dueño Directo" if x else "Inmobiliaria" for x in dfSinVendedor.user_id.isin(idxDuenos)]

# Asignacion de los valores calculados para tipo de vendedor
dfFiltradoSet = dfFiltradoSet.set_index(['idregistro'])\
    .combine_first(dfSinVendedor[['idregistro', 'tipo_vendedor']].set_index(['idregistro']))\
    .reset_index()

# Verificacion del tipo de vendedor
srFreqParqueo = pd.Series(dfFiltradoSet['tipo_vendedor']).value_counts(dropna = False)
srFreqParqueo

Inmobiliaria     526
Dueño Directo     31
Name: tipo_vendedor, dtype: int64

### Data Normalization

El objetivo es normalizar  la columna de espacio que tiene una escala mayor a diferencia de las otras variables. Para la normalizacion se utilizara Z score.

In [311]:
dfFiltradoSet['espacio_m2'] = (dfFiltradoSet['espacio_m2'] - dfFiltradoSet['espacio_m2'].min()) / (dfFiltradoSet['espacio_m2'].max() - dfFiltradoSet['espacio_m2'].min())

dfFiltradoSet

,idregistro,banos,espacio_m2,habitaciones,latitud,longitud,moneda,parqueo,precio_real,tipo_vendedor,ubicacion,user_id
0,1007688385,3.0,0.358714,3.0,14.591,-90.506,US$,1,2299310.00,Inmobiliaria,U0,55074295
1,1014321966,2.0,0.098139,1.0,14.600,-90.511,US$,1,1207330.00,Inmobiliaria,U0,34408653
2,1014324725,2.0,0.076142,2.0,14.593,-90.491,US$,1,922800.00,Inmobiliaria,U0,34408653
3,1023944274,2.0,0.091371,2.0,14.588,-90.503,US$,1,1647474.84,Inmobiliaria,U0,24012106
4,1029720345,2.0,0.456853,3.0,14.607,-90.470,US$,1,5190750.00,Inmobiliaria,U5,24012106
...,...,...,...,...,...,...,...,...,...,...,...,...
552,1100424578,2.0,0.250423,3.0,14.591,-90.493,US$,1,2378486.24,Inmobiliaria,U0,25529822
553,1100424611,3.0,0.656514,3.0,14.591,-90.493,US$,1,5136920.00,Inmobiliaria,U0,25529822
554,1100424722,3.0,0.186125,3.0,14.621,-90.516,US$,0,1768700.00,Inmobiliaria,U3,4263970
555,1100425243,2.0,0.240271,3.0,14.634,-90.507,US$,1,2153200.00,Inmobiliaria,U3,38828922


### Hot Encoding

Vamos a crear dummy variables para aquellas variables que son de tipo categorico. Primero verificaremos los tipos de datos del actual dataframe.

In [312]:
dfFiltradoSet.dtypes

idregistro        object
banos            float64
espacio_m2       float64
habitaciones     float64
latitud          float64
longitud         float64
moneda            object
parqueo            int64
precio_real      float64
tipo_vendedor     object
ubicacion         object
user_id            int64
dtype: object

In [313]:
# Eliminamos variables dummy si ya existen
dfFiltradoSet = dfFiltradoSet.drop(['moneda_q','moneda_d', 'tipo_inmobiliaria', 'tipo_dueno'], axis=1, errors='ignore')

# Convertimos aquellas variables categoricas a variables dummy.
dfFiltradoSet = pd.concat([dfFiltradoSet, pd.get_dummies(dfFiltradoSet['moneda'])\
                              .rename(columns={'Q': 'moneda_q', 'US$': 'moneda_d'})], axis = 1)

dfFiltradoSet = pd.concat([dfFiltradoSet, pd.get_dummies(dfFiltradoSet['tipo_vendedor'])\
                              .rename(columns={'Inmobiliaria': 'tipo_inmobiliaria', 
                                               'Dueño Directo': 'tipo_dueno'})], axis = 1)

dfFiltradoSet = pd.concat([dfFiltradoSet, pd.get_dummies(dfFiltradoSet['ubicacion'])], axis = 1)

# Verificamos las variables resultantes
dfFiltradoSet.dtypes

idregistro            object
banos                float64
espacio_m2           float64
habitaciones         float64
latitud              float64
longitud             float64
moneda                object
parqueo                int64
precio_real          float64
tipo_vendedor         object
ubicacion             object
user_id                int64
moneda_q               uint8
moneda_d               uint8
tipo_dueno             uint8
tipo_inmobiliaria      uint8
U0                     uint8
U2                     uint8
U3                     uint8
U4                     uint8
U5                     uint8
U6                     uint8
U7                     uint8
U8                     uint8
dtype: object

### Seleccion Variables Modelo

In [315]:
# Seleccion de las variables
dfModeloSet = dfFiltradoSet[['idregistro', 'banos', 'espacio_m2', 
                               'habitaciones', 'moneda_q', 'moneda_d',
                               'parqueo', 'tipo_dueno', 'tipo_inmobiliaria', 
                               'U0', 'U2', 'U3', 'U4', 'U5', 'U6', 'U7', 'U8',
                               'precio_real'
                              ]]

# Verificacion de las variables
dfModeloSet.dtypes

idregistro            object
banos                float64
espacio_m2           float64
habitaciones         float64
moneda_q               uint8
moneda_d               uint8
parqueo                int64
tipo_dueno             uint8
tipo_inmobiliaria      uint8
U0                     uint8
U2                     uint8
U3                     uint8
U4                     uint8
U5                     uint8
U6                     uint8
U7                     uint8
U8                     uint8
precio_real          float64
dtype: object

## Construccion Modelo

In [324]:
# Seleccionamos las variables predictoras
dfPredictors = dfModeloSet.loc[:, dfModeloSet.columns != 'precio_real']

# Seleccionamos la variable dependiente
Y = dfModeloSet['precio_real']

# Separamos los datos para test y training
xTrain, xTest, yTrain, yTest = train_test_split(dfPredictors, Y, test_size = 0.3, random_state=1505)

xTrain

,idregistro,banos,espacio_m2,habitaciones,moneda_q,moneda_d,parqueo,tipo_dueno,tipo_inmobiliaria,U0,U2,U3,U4,U5,U6,U7,U8
352,1100385824,1.0,0.025381,1.0,1,0,1,0,1,0,0,1,0,0,0,0,0
411,1100402190,2.0,0.152284,2.0,0,1,0,0,1,0,0,0,0,1,0,0,0
82,1066544531,4.0,0.561760,4.0,0,1,1,0,1,1,0,0,0,0,0,0,0
301,1100373467,2.0,0.165821,3.0,1,0,1,0,1,0,0,1,0,0,0,0,0
494,1100413522,2.0,0.128596,3.0,0,1,0,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,1100368627,2.0,0.099831,2.0,1,0,1,1,0,0,0,0,1,0,0,0,0
127,1100154670,2.0,0.131980,3.0,0,1,1,0,1,0,0,0,1,0,0,0,0
425,1100403742,4.0,0.175973,3.0,0,1,1,0,1,1,0,0,0,0,0,0,0
408,1100402157,3.0,0.094755,2.0,0,1,1,0,1,1,0,0,0,0,0,0,0
